# Haciendo la sintonización fina de BERT para el dataset

El notebook esta diseñado para correr en colab, por lo que se incluyeron los comandos de instalación para librerías no presentes por defecto

In [3]:
!pip install transformers[torch]
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import datasets
from transformers import (AutoTokenizer, DataCollatorWithPadding, AutoConfig, AutoModel,
                          AutoModelForSequenceClassification, BertConfig, Trainer, BertForSequenceClassification,)
from transformers import TrainingArguments
import evaluate
import torch.nn as nn
import torch
import numpy as np

In [ ]:
checkpoint = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

VOCAB = 21
#Limitamos el vocab_size = 22 (tipos de productos) y una secuencia máxima de 1024 productos, además de definir 13 etiquetas de salida
config = BertConfig(vocab_size=VOCAB, max_position_embeddings=1600, pad_token_id=13, num_labels=VOCAB-1)
config

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_15": 15,
    "LABEL_16": 16,
    "LABEL_17": 17,
    "LABEL_18": 18,
    "LABEL_19": 19,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
  

In [ ]:
model = AutoModelForSequenceClassification.from_config(config)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21, 768, padding_idx=13)
      (position_embeddings): Embedding(1600, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
#Prueba para ver como es que el modelo saca las predicciones
input = torch.tensor([[0, 0, 1, 2, 2, 3, 13]]).to(device)
softmax = torch.nn.Softmax(dim=1)
softmax(model(input)[0]), model(input)[0][0]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


(tensor([[0.0876, 0.0449, 0.0545, 0.0403, 0.0454, 0.0599, 0.0586, 0.0385, 0.0558,
          0.0356, 0.0485, 0.0446, 0.0509, 0.0403, 0.0359, 0.0500, 0.0528, 0.0405,
          0.0730, 0.0424]], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([ 0.2326,  0.0870, -0.0133,  0.1511,  0.0923,  0.1820,  0.3142, -0.1185,
          0.2157, -0.0911, -0.3076,  0.2358, -0.0413, -0.1051,  0.0088, -0.1614,
          0.1344, -0.0027,  0.2269, -0.2325], device='cuda:0',
        grad_fn=<SelectBackward0>))

In [6]:
#Cargamos el dataset
df = pd.read_hdf('data_piezas5.h5', key='df')
df['X_tuple'] = df.X.apply(tuple)
df['y_tuple'] = df.y.apply(tuple)
df = df.drop_duplicates(subset=['X_tuple', 'y_tuple']) #Eliminamos los datos replicados X->y
df = df.drop(labels=['FE_EMIT', 'FECHA_INICIO', 'CG_PROD', 'X_tuple', 'y_tuple'], axis=1) #Eliminamos columnas que no son útiles
df['y'] = df['y'].apply(lambda x: x[0])
df

,X,y
12027,"[12, 1, 0, 12, 12, 0, 12, 0, 12, 0, 3, 5, 0, 1...",12
12014,"[12, 1, 10, 0, 0, 12, 8, 5, 0, 12, 0, 0, 12, 0...",12
12011,"[4, 2, 1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0,...",4
12010,"[4, 2, 1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0,...",2
12008,"[1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0, 0, 12...",1
...,...,...
17,"[2, 13, 13, 3, 5, 8, 5, 8, 2, 10, 12, 5, 5, 2,...",10
1,"[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2]",2
2,"[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2]",12
3,"[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2]",10


In [ ]:
#Vemos la distribución de clases del dataframe
df.y.value_counts()

In [ ]:
#Definimos funciones que son necesarias para el input de BERT, consultese: https://huggingface.co/learn/nlp-course/chapter3/2?fw=pt para más información
from typing import Union
#Función para padear las secuencias
def pad(max_len:int, sequence:Union[list, np.array], pad_token:int=20):
  padding = np.full(max_len-len(sequence), pad_token)
  padded_seq = np.concatenate((sequence, padding))
  return padded_seq.tolist()

#Función para crear las máscaras de atención
def attention_mask(pad_sequence:Union[list, np.array], pad_token:int=20):
  seq = np.asarray(pad_sequence, dtype=int)
  attention_mask = np.asarray(seq != pad_token, dtype=int)
  return attention_mask.tolist()

#Función para crear los arreglos que especifican el tipo de token
def token_types(pad_sequence:Union[list, np.array]):
  return np.zeros(len(pad_sequence), dtype=int).tolist()

In [ ]:
#Preparamos el df para BERT con las columnas que requiere('idx', 'label', 'input_ids', 'attention_mask', 'token_type_ids')
max_len = df.X.apply(lambda x: len(x))
max_len = max(max_len)
dt = df.copy(True)
PAD = 20

dt.insert(0, 'idx', np.full(len(df), np.nan))
dt.insert(1, 'label', dt.y)
dt.insert(2, 'input_ids', np.full(len(df), np.nan))
dt.insert(3, 'attention_mask', np.full(len(df), np.nan))
dt.insert(4, 'token_type_ids', np.full(len(df), np.nan))
data = dt.to_numpy()

for i in range(len(data)):
  data[i][2] = pad(max_len, data[i][5], pad_token=PAD)
  data[i][3] = attention_mask(data[i][2], pad_token=PAD)
  data[i][4] = token_types(data[i][2])
dt['input_ids'] = data[:, 2]
dt['attention_mask'] = data[:, 3]
dt['token_type_ids'] = data[:, 4]
df = dt.copy(True)
df

,idx,label,input_ids,attention_mask,token_type_ids,X,y
12027,NaN,12,"[12, 1, 0, 12, 12, 0, 12, 0, 12, 0, 3, 5, 0, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 1, 0, 12, 12, 0, 12, 0, 12, 0, 3, 5, 0, 1...",12
12014,NaN,12,"[12, 1, 10, 0, 0, 12, 8, 5, 0, 12, 0, 0, 12, 0...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[12, 1, 10, 0, 0, 12, 8, 5, 0, 12, 0, 0, 12, 0...",12
12011,NaN,4,"[4, 2, 1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 2, 1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0,...",4
12010,NaN,2,"[4, 2, 1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 2, 1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0,...",2
12008,NaN,1,"[1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0, 0, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 19, 5, 10, 0, 0, 12, 8, 5, 0, 12, 0, 0, 12...",1
...,...,...,...,...,...,...,...
9,NaN,5,"[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2, 20, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2]",5
10,NaN,5,"[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2, 20, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2]",5
11,NaN,10,"[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2, 20, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2]",10
6,NaN,2,"[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2, 20, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 12, 10, 5, 5, 12, 2, 5, 5, 10, 2, 2]",2


In [ ]:
dt = df.sample(frac = 1).reset_index(drop=True) #Mezclamos el df
# El split es 80/20, sin dataset de testeo
train = int(len(dt)*0.8)
tdf = dt[:train]
vdf = dt[train:]
#Agregamos finalmente el idx a cada elemento
tdf['idx'] = tdf.reset_index(drop=True).index.values
vdf['idx'] = vdf.reset_index(drop=True).index.values
tdf

<ipython-input-41-96401a3fa5dd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdf['idx'] = tdf.reset_index(drop=True).index.values
<ipython-input-41-96401a3fa5dd>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vdf['idx'] = vdf.reset_index(drop=True).index.values


,idx,label,input_ids,attention_mask,token_type_ids,X,y
0,0,5,"[2, 1, 12, 12, 2, 5, 2, 3, 4, 0, 3, 2, 3, 3, 3...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 12, 12, 2, 5, 2, 3, 4, 0, 3, 2, 3, 3, 3...",5
1,1,1,"[5, 10, 10, 0, 1, 2, 2, 2, 2, 2, 8, 8, 3, 5, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 10, 10, 0, 1, 2, 2, 2, 2, 2, 8, 8, 3, 5, 2...",1
2,2,4,"[4, 3, 3, 3, 3, 2, 2, 5, 2, 2, 2, 2, 0, 4, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 3, 3, 3, 3, 2, 2, 5, 2, 2, 2, 2, 0, 4, 0, ...",4
3,3,2,"[5, 12, 12, 4, 3, 0, 3, 0, 2, 5, 12, 2, 0, 8, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 12, 12, 4, 3, 0, 3, 0, 2, 5, 12, 2, 0, 8, ...",2
4,4,2,"[2, 0, 12, 2, 2, 2, 4, 4, 3, 3, 3, 4, 1, 2, 5,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 0, 12, 2, 2, 2, 4, 4, 3, 3, 3, 4, 1, 2, 5,...",2
...,...,...,...,...,...,...,...
9471,9471,0,"[2, 2, 5, 3, 0, 2, 3, 5, 13, 10, 10, 2, 17, 17...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 5, 3, 0, 2, 3, 5, 13, 10, 10, 2, 17, 17...",0
9472,9472,5,"[8, 1, 4, 5, 7, 2, 3, 8, 3, 2, 4, 5, 4, 5, 4, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[8, 1, 4, 5, 7, 2, 3, 8, 3, 2, 4, 5, 4, 5, 4, ...",5
9473,9473,2,"[2, 2, 5, 2, 2, 3, 8, 8, 8, 2, 8, 8, 19, 5, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2, 2, 5, 2, 2, 3, 8, 8, 8, 2, 8, 8, 19, 5, 2,...",2
9474,9474,4,"[4, 4, 3, 5, 3, 4, 2, 2, 4, 13, 5, 5, 13, 5, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 4, 3, 5, 3, 4, 2, 2, 4, 13, 5, 5, 13, 5, 5...",4


In [ ]:
#Se pasa todo a un datasetdict porque así trabajan los modelos de HuggingFace
tds = Dataset.from_pandas(tdf)
vds = Dataset.from_pandas(vdf)

ds = DatasetDict()

ds['train'] = tds
ds['validation'] = vds

ds

DatasetDict({
    train: Dataset({
        features: ['idx', 'label', 'input_ids', 'attention_mask', 'token_type_ids', 'X', 'y'],
        num_rows: 9476
    })
    validation: Dataset({
        features: ['idx', 'label', 'input_ids', 'attention_mask', 'token_type_ids', 'X', 'y'],
        num_rows: 2369
    })
})

In [ ]:
#Especificamos los argumentos de entrenamiento
training_args = TrainingArguments("test-trainer",
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=24,
                                  num_train_epochs=17,
                                  learning_rate = 3e-05,
                                  evaluation_strategy="epoch",
                                  save_steps = 1500,
                                  )

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    ord = np.argsort(logits, axis=-1) #Ordenamos los índices de los logits de menor a mayor
    sec_pred = ord[:, -2]
    third_pred = ord[:, -3]
    accuracy = metric.compute(predictions=predictions, references=labels)['accuracy']
    accuracy += metric.compute(predictions=sec_pred, references=labels)['accuracy']
    accuracy += metric.compute(predictions=third_pred, references=labels)['accuracy']
    acc = {'accuracy':accuracy}
    return acc

In [ ]:
#Testeamos la función de precisión definida
eval = np.array([[-1, 2, 4],[2, 3, -2],[-5, 9, -10]]), [0, 1, 2]
print(compute_metrics(eval))

{'accuracy': 1.0}


In [ ]:
#Creamos un CustomTrainer que deriva del Trainer, ya que el costo del Trainer base lanza errores
class CustomTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            outputs = model(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                token_type_ids=inputs['token_type_ids']
            )
            labels = torch.zeros(outputs['logits'].shape)
            labels[0, inputs['labels']] = 1
            loss = nn.BCEWithLogitsLoss()(outputs['logits'].to(device),
                                             labels.to(device))
            return (loss, outputs) if return_outputs else loss

In [ ]:
#Inicializamos el CustomTrainer
trainer = CustomTrainer(
    model,
    training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.168400,0.106487,0.495146


# Etapa de evaluación de modelo

In [ ]:
y_train = trainer.predict(ds["train"])
print(y_train.predictions.shape, y_train.label_ids.shape)

(3954, 20) (3954,)


In [ ]:
y_test = trainer.predict(ds["validation"])
print(y_test.predictions.shape, y_test.label_ids.shape)

(989, 20) (989,)


In [ ]:
# Get the top 3 predicted class indices for each instance in the training set
train_top3_preds = np.argsort(y_train.predictions, axis=-1)[:, -3:]

# Get the top 3 predicted class indices for each instance in the test set
test_top3_preds = np.argsort(y_test.predictions, axis=-1)[:, -3:]

In [ ]:
import evaluate
# For the training set
train_accuracy = np.mean(np.any(train_top3_preds == y_train.label_ids.reshape(-1, 1), axis=-1))

# For the test set
test_accuracy = np.mean(np.any(test_top3_preds == y_test.label_ids.reshape(-1, 1), axis=-1))

# Print or use train_accuracy and test_accuracy as needed
print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

Training Accuracy: 0.47470915528578655
Test Accuracy: 0.4398382204246714


In [ ]:
import torch
# Save your model to Google Drive
torch.save(model.state_dict(), '/content/drive/MyDrive/BERT PIEZAS/model_piezas.pth')

In [ ]:
from google.colab import files

# Compress the model and tokenizer files into a zip archive (optional)
!zip -r model.zip /content/drive/MyDrive/BERT PIEZAS/model_piezas.pth

# Download the zip file to your local machine (optional)
files.download('/content/drive/MyDrive/BERT PIEZAS/model_piezas.zip')

	zip warning: name not matched: /content/drive/MyDrive/BERT
	zip warning: name not matched: PIEZAS/model_piezas.pth

zip error: Nothing to do! (try: zip -r model.zip . -i /content/drive/MyDrive/BERT PIEZAS/model_piezas.pth)


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
